# Welcome to Dash

### UI elements not quite as messy as ipywidgets.

Dash is a comparably easy and stable way to build standalone or jupyter based widgets/apps.

All components listed here can also be found under https://dash.plotly.com/dash-core-components for a more specific documentation.


Simple text is automatically shown side by side: <br>
`html.Div(["some nested text ",   "some parallel text"]),`<br>
while nested Div's are shown side by side:<br>
`html.Div([html.Div("some nested text "),   html.Div("some parallel text")])`<br>
Unless you use the style property to change the display property to inline-block: <br> 
`html.Div([html.Div("some nested text   ", style={"display":"inline-block"}),   html.Div("some parallel text", style={"display":"inline-block"})],),`


In [1]:
import plotly.express as px
import numpy as np

In [2]:
from dash import html, Input, Output, dcc, State, Dash
import jupyter_dash


In [3]:
# the UI is defined as a hierarchy of HTML components inside 
# the app.layout
basic_layout = html.Div(
    [
        html.Div(["some nested text ", "some parallel text"]),
        html.Br(),
        html.Div([html.Div("some nested text "), 
                  html.Div("some parallel text")]),
        html.Br(),
        html.Div(
            [
                html.Div(
                    "some nested text   ",
                    style={"display": "inline-block"},
                ),
                html.Div(
                    "some parallel text",
                    style={"display": "inline-block"},
                ),
            ],
        ),
    ]
)

In [4]:
# standalone dash server
app1 = Dash("app1")

app1.layout = basic_layout
#app1.run_server(debug=True, port=5050, use_reloader=False)

In [5]:
# dash as jupyter widgets
app2 = jupyter_dash.JupyterDash("app2")

app2.layout = basic_layout

app2.run_server(debug=True, port=8069, mode="inline")

Dash is running on http://127.0.0.1:8069/



# Styling and components
With the `style` argument most dash components can be changed according to the css standard.

Note: Dash also supports css style sheets. See: https://dash.plotly.com/external-resources


Most dash components are found under `dcc`, though some are in `html`.
With these we can generate a UI that can't really do anything.

In [6]:
external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]
app3 = jupyter_dash.JupyterDash(
        "app3", 
        external_stylesheets=external_stylesheets)
my_style = {"width": "30%", 
            "margin-top": "20px", 
            "margin-bottom": "20px"}
app3.layout = html.Div(
    ["This is some basic layout:",
        dcc.Dropdown(
            ["x^2", "2x", "e^x"],
            "x^2",
            style=my_style,
        ),
        html.Div(dcc.RangeSlider(
             0, 20, 1, 
             value=[5, 15]), 
             style={"width": "30%"}),
        html.Button(
            "Click_me",
            style=my_style,
        ),
        dcc.Graph(),
    ]
)

In [7]:
app3.run_server(debug=True, port=8070, mode="inline")

Dash is running on http://127.0.0.1:8070/



# Callbacks
With the use of callbacks we can now add functionality to all our elements.

In this example I want to be able to choose a function type, set the x limits for the calculation and show the graph upon clicking the button.

The Dash callbacks allow us to access and monitor each object variable. <br>
For this to work we first need to assign IDs to every object we want to interact with.  <br>
Many of the Dividers for example don't need a specific ID. <br>

A callback can have as many inputs and outputs as needed. <br>
Any component provided as `Input` will trigger the callback, while `State` can be used to obtain certain variables without triggering the function. <br>
Lastly `Output` is used to define which object the return value will be assigned to.


Note that even though the `n_click` value of the button is not used it must still be the first function argument.

In [8]:
app4 = jupyter_dash.JupyterDash("app4")

app4.layout = html.Div(
    [
        "This is some basic layout:",
        dcc.Dropdown(
            ["x^2", "2x", "e^x"],
            "x^2",
            style=my_style,
            id="dropdown",
        ),
        html.Div(
            dcc.RangeSlider(
                0,
                20,
                1,
                value=[5, 15],
                id="slider",
            ),
            style={"width": "50%"},
        ),
        html.Button(
            "Click_me",
            style=my_style,
            id="button",
        ),
        dcc.Graph(id="graph"),
    ]
)

In [9]:

@app4.callback(
    Output("graph", "figure"),
    Input("button", "n_clicks"),
    State("dropdown", "value"),
    State("slider", "value"),
)
def update_graph(n_clicks, dropdown_value, slider_value):
    def _plot_function(x, function_name):
        if function_name == "x^2":
            return x**2
        elif function_name == "2x":
            return 2 * x
        elif function_name == "e^x":
            return np.exp(x)
        else:
            raise ValueError(
                f"Unknown function_name: {function_name}")

    x_range = np.linspace(slider_value[0], slider_value[1], 100)
    y = _plot_function(x_range, dropdown_value)
    figure = px.line(x=x_range, y=y, title=dropdown_value)

    return figure




In [10]:
# what happens if I deselect the functions?


In [23]:
app4.run_server(debug=True, port=8068, mode="inline")

Dash is running on http://127.0.0.1:8068/

Dash app running on http://127.0.0.1:8068/


# Dynamically add more widgets

So far we have only considered static IDs and that is fine for many work cases. However sometimes it might be necessary to add widgets inside of callbacks.
An example for this could be the creation of a new tab with its own button and text on the inside.


For these callbacks dash provides three patterns `MATCH` `ALL` and `AllSMALLER`.
Here I will only go over `MATCH`, for more information see https://dash.plotly.com/pattern-matching-callbacks

In [12]:
from dash.dependencies import MATCH

app5 = jupyter_dash.JupyterDash("app5", external_stylesheets=external_stylesheets)

app5.layout = html.Div(
    [
        html.Button("Add Tab", id="button_add_tab"),
        dcc.Tabs(id="tabs", children=[]),
    ]
)





In [13]:
@app5.callback(
    Output("tabs", "children"),
    Input("button_add_tab", "n_clicks"),
    State("tabs", "children"),
    prevent_initial_call=True,
)
def add_tab(n_clicks, tabs_children):  
    new_tab = dcc.Tab(
        label=f"Tab {n_clicks}",
        children=[
            html.Div(
                [
                    html.Button(
                        f"Button {n_clicks}",
                        id={"type": "button_tab", "index": n_clicks},
                    ),
                    html.Div(
                        f"Button {n_clicks} clicked 0 times. ",
                        id={"type": "div_tab", "index": n_clicks},
                    ),
                ]
            )
        ],
    )
    tabs_children.append(new_tab)
    return tabs_children



In [14]:

@app5.callback(
    Output({"type": "div_tab", "index": MATCH}, "children"),
    Input({"type": "button_tab", "index": MATCH}, "n_clicks"),
    State({"type": "button_tab", "index": MATCH}, "id"),
    prevent_initial_call=True,
)
def tabs_button_click(n_clicks, button_id):
    return f"Button {button_id['index']} clicked {n_clicks} times. "



In [15]:
app5.run_server(debug=True, port=8078, mode="inline")

Dash is running on http://127.0.0.1:8078/



# Using dash inside a class

Unfortunately the decorator style of dashs callbacks we have used so far is very much incompatible with encapsulating the dash app inside a class. 
Normally the app itself is to be used in the whole module. <br>
`@self.app.callback()` or smililar things don't work.
However we can simply refer any function as a calback as seen here:

##### A piece of warning though: 
The [dash website](https://dash.plotly.com/sharing-data-between-callbacks) advices against using a callback to access out of scope data or variables. As far as I can tell this is only relevant when deploying the dash server in a way that multiple user access the same instance and it should not be a problem for local or cloud hosted python environments.






In [16]:
class App6:
    def __init__(self):
        external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]
        self.app6 = jupyter_dash.JupyterDash(
            "app6", external_stylesheets=external_stylesheets
        )
        my_style = { "width": "30%",
                        "margin-top": "20px",
                        "margin-bottom": "20px",}
        self.app6.layout = html.Div(
            [
                "This is some basic layout:",
                dcc.Dropdown(
                    options=["x^2", "2x", "e^x"],
                    # options = {"x^2":"quadratic", "2x":"linear", "e^x":"exponential"}, 

                    placeholder="Select a function",
                    value = "x^2",

                    style=my_style,
                    id="dropdown",
                ),
                html.Div(
                    dcc.RangeSlider(0, 20, 1, value=[5, 15], id="slider"),
                    style={"width": "50%"},
                ),
                html.Button(
                    "Click_me",
                    style=my_style,
                    id="button",
                ),
                dcc.Graph(id="graph"),
            ]
        )
        self.app6.callback(
            Output("graph", "figure"),
            Input("button", "n_clicks"),
            State("dropdown", "value"),
            State("slider", "value"),
        )(self.update_graph)

    def update_graph(self, n_clicks, dropdown_value, slider_value):
        def _plot_function(x, function_name):
            if function_name == "x^2":
                return x**2
            elif function_name == "2x":
                return 2 * x
            elif function_name == "e^x":
                return np.exp(x)
            elif function_name == None:
                return None
            else:
                raise ValueError(f"Unknown function_name: {function_name}, type: {type(function_name)}")

        x_range = np.linspace(slider_value[0], slider_value[1], 100)
        y = _plot_function(x_range, dropdown_value)
        if y is not None:
            figure = px.line(x=x_range, y=y, title=dropdown_value)
        else:
            figure = px.line()

        return figure

    def run(self, port=8081):
        self.app6.run_server(debug=True, port=port, mode="inline")

In [17]:
app_6 = App6()
app_6.run(port=8088)

Dash is running on http://127.0.0.1:8088/



# Additional Dash Components

- [Download button](https://dash.plotly.com/dash-core-components/download)
- [Upload button](https://dash.plotly.com/dash-core-components/upload)
- [Data Tables from pandas](https://dash.plotly.com/datatable)
- [Bio and molecule viewer](https://dash.plotly.com/dash-bio)
- [Many more]( https://dash.plotly.com/)

# Extendet dash functionality 


- [Dash Extensions - Enrich](https://www.dash-extensions.com/getting_started/enrich)
- [Dash json viwer](https://github.com/ghandic/dash_renderjson)

## Dash extensions DashBlueprint

Blueprints can be used to create and plan dash layouts and callbacks. Because these blueprints do not call the DashApp directly they can be created in differend scopes, files or libraries and later imported when needed. 
This can help keep the actual code much cleaner.

In [18]:
from dash_extensions.enrich import DashBlueprint, DashProxy#, html, Output, Input
bp = DashBlueprint()


bp.layout = html.Div(
    [
        "This is some basic layout:",
        dcc.Dropdown(
            ["x^2", "2x", "e^x"],
            "x^2",
            style=my_style,
            id="dropdown",
        ),
        html.Div(
            dcc.RangeSlider(
                0,
                20,
                1,
                value=[5, 15],
                id="slider",
            ),
            style={"width": "50%"},
        ),
        html.Button(
            "Click_me",
            style=my_style,
            id="button",
        ),
        dcc.Graph(id="graph"),
    ]
)


In [21]:
@bp.callback(
    Output("graph", "figure"),
    Input("button", "n_clicks"),
    State("dropdown", "value"),
    State("slider", "value"),
)
def update_graph2(n_clicks, dropdown_value, slider_value):
    def _plot_function(x, function_name):
        if function_name == "x^2":
            return x**2
        elif function_name == "2x":
            return 2 * x
        elif function_name == "e^x":
            return np.exp(x)
        else:
            raise ValueError(f"Unknown function_name: {function_name}")

    x_range = np.linspace(slider_value[0], slider_value[1], 100)
    y = _plot_function(x_range, dropdown_value)
    figure = px.line(x=x_range, y=y, title=dropdown_value)

    return figure


In [22]:
app7 = DashProxy(blueprint=bp)
#app7.run_server()

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Input In [9], in update_graph.<locals>._plot_function(
    x=array([ 5.        ,  5.1010101 ,  5.2020202 ,  5...6969697 , 14.7979798 , 14.8989899 , 15.        ]),
    function_name=None
)
     14     return np.exp(x)
     15 else:
---> 16     raise ValueError(f"Unknown function_name: {function_name}")

ValueError: Unknown function_name: None

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Input In [9], in update_graph.<locals>._plot_function(
    x=array([ 5.        ,  5.1010101 ,  5.2020202 ,  5...6969697 , 14.7979798 , 14.8989899 , 15.        ]),
    function_name=None
)
     14     return np.exp(x)
     15 else:
---> 16     raise ValueError(f"Unknown function_name: {function_name}")

ValueError: Unknown function_name: None

------

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Input In [9], in update_graph.<locals>._plot_function(
    x=array([ 5.        ,  5.1010101 ,  5.2020202 ,  5...6969697 , 14.7979798 , 14.8989899 , 15.        ]),
    function_name=None
)
     14     return np.exp(x)
     15 else:
---> 16     raise ValueError(f"Unknown function_name: {function_name}")

ValueError: Unknown function_name: None

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Input In [9], in update_graph.<locals>._plot_function(
    x=array([ 5.        ,  5.1010101 ,  5.2020202 ,  5...6969697 , 14.7979798 , 14.8989899 , 15.        ]),
    function_name=None
)
     14     return np.exp(x)
     15 else:
---> 16     raise ValueError(f"Unknown function_name: {function_name}")

ValueError: Unknown function_name: None

------

The problem here is that DashProxy and JupyterDash are not compatible.
If you run `DashProxy.run_server()` in a notebook the cell will never finish. 

# Personal grievances

I have two problems with dash that I have not found a good solution for.

-  First is that dash code can get very convoluted and messy. <br>
   Extracting part of the layout into individual functions can help a lot, but it still mostly looks messy.

- Second is Dash's tendency to swallow error messages, especially inside a notebook. <br>
This can be somewhat circumvented by running dash in a browser as that at least provides you with some of the messages. But mostly its just annoying.<br>
Also printing and logging doesn't always work either
